In [32]:
dow_jones = [
    "AAPL",  # Apple Inc.
    "AMGN",  # Amgen Inc.
    "AXP",   # American Express Company
    "BA",    # Boeing Company
    "CAT",   # Caterpillar Inc.
    "CRM",   # Salesforce, Inc.
    "CSCO",  # Cisco Systems, Inc.
    "CVX",   # Chevron Corporation
    "DIS",   # Walt Disney Company
    "DOW",   # Dow Inc.
    "GS",    # Goldman Sachs Group, Inc.
    "HD",    # Home Depot, Inc.
    "HON",   # Honeywell International Inc.
    "IBM",   # International Business Machines Corporation
    "INTC",  # Intel Corporation
    "JNJ",   # Johnson & Johnson
    "JPM",   # JPMorgan Chase & Co.
    "KO",    # Coca-Cola Company
    "MCD",   # McDonald's Corporation
    "MMM",   # 3M Company
    "MRK",   # Merck & Co., Inc.
    "MSFT",  # Microsoft Corporation
    "NKE",   # NIKE, Inc.
    "PG",    # Procter & Gamble Company
    "TRV",   # Travelers Companies, Inc.
    "UNH",   # UnitedHealth Group Incorporated
    "V",     # Visa Inc.
    "VZ",    # Verizon Communications Inc.
    "WBA",   # Walgreens Boots Alliance, Inc.
    "WMT"    # Walmart Inc.
]

In [33]:
import yfinance as yf
import pandas as pd

# 定义资产列表
tickers = dow_jones

# 下载调整后的收盘价（过去 1 年）
raw_df = yf.download(tickers, start="2006-03-27", end="2025-03-27", interval="1d")

# 检查数据
print(raw_df.head())

[*********************100%***********************]  30 of 30 completed

Price          Close                                                        \
Ticker          AAPL       AMGN        AXP         BA        CAT       CRM   
Date                                                                         
2006-03-27  1.790871  50.995193  39.942829  56.011147  46.315220  9.519331   
2006-03-28  1.766796  50.653389  39.351482  55.382359  46.001884  9.173760   
2006-03-29  1.875734  50.241776  39.299076  56.575619  45.264568  9.119065   
2006-03-30  1.888373  50.660358  39.022121  56.032589  44.484264  9.153872   
2006-03-31  1.887471  50.751041  39.336514  55.682468  44.121738  9.032053   

Price                                            ...   Volume            \
Ticker           CSCO        CVX        DIS DOW  ...      MRK      MSFT   
Date                                             ...                      
2006-03-27  14.375547  28.175625  22.160187 NaN  ...  6115709  59908600   
2006-03-28  14.136946  28.006205  21.957554 NaN  ...  6691899  58520500   


In [34]:
raw_df.head()

Price          Close                                                        \
Ticker          AAPL       AMGN        AXP         BA        CAT       CRM   
Date                                                                         
2006-03-27  1.790871  50.995193  39.942829  56.011147  46.315220  9.519331   
2006-03-28  1.766796  50.653389  39.351482  55.382359  46.001884  9.173760   
2006-03-29  1.875734  50.241776  39.299076  56.575619  45.264568  9.119065   
2006-03-30  1.888373  50.660358  39.022121  56.032589  44.484264  9.153872   
2006-03-31  1.887471  50.751041  39.336514  55.682468  44.121738  9.032053   

Price                                            ...   Volume            \
Ticker           CSCO        CVX        DIS DOW  ...      MRK      MSFT   
Date                                             ...                      
2006-03-27  14.375547  28.175625  22.160187 NaN  ...  6115709  59908600   
2006-03-28  14.136946  28.006205  21.957554 NaN  ...  6691899  58520500   
2006-03-29  14.296004  28.378920  22.581671 NaN  ...  8603346  53150300   
2006-03-30  14.561118  28.320824  22.589773 NaN  ...  7737908  54612000   
2006-03-31  14.362288  28.059452  22.605986 NaN  ...  9235605  62190500   

Price                                                                     \
Ticker           NKE        PG      TRV       UNH   V        VZ      WBA   
Date                                                                       
2006-03-27   8194400   8040400  3301800  11124700 NaN  11987465  4527400   
2006-03-28   8464000  14113000  3601200  12203500 NaN  10665359  5734800   
2006-03-29   6694400  12430600  4016900   9507000 NaN   8699658  3052700   
2006-03-30  11872000   8419500  2974300   8091100 NaN  14713183  3189000   
2006-03-31   8536800  11176100  2454500   9771000 NaN  20164062  3325900   

Price                 
Ticker           WMT  
Date                  
2006-03-27  25991400  
2006-03-28  30132900  
2006-03-29  26045400  
2006-03-30  29380800  
2006-03-31  30273000  

[5 rows x 150 columns]

In [17]:
#pd.read_parquet('raw_df.parquet')

In [35]:
raw_df.columns = raw_df.columns.swaplevel('Price','Ticker')

In [36]:
raw_df.columns.levels[1]

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')

In [37]:
assert isinstance(raw_df, pd.DataFrame)
assert isinstance(raw_df.index, pd.DatetimeIndex)
assert isinstance(raw_df.columns, pd.MultiIndex)

In [38]:
from deepdow.utils import raw_to_Xy

n_timesteps = len(raw_df)  # 20
n_channels = len(raw_df.columns.levels[1])  # 2
n_assets = len(raw_df.columns.levels[0])  # 2

lookback, gap, horizon = 90, 2, 30

X, timestamps, y, asset_names, indicators = raw_to_Xy(raw_df,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      freq="B",
                                                      horizon=horizon)

n_samples =  n_timesteps - lookback - horizon - gap + 1  # 10

# assert X.shape == (n_samples, n_channels, lookback, n_assets)
# assert timestamps[0] == raw_df.index[lookback]
# assert asset_names == tickers
# assert indicators == ['Close', 'High', 'Low', 'Open', 'Volume']

In [39]:
from deepdow.data import InRAMDataset
import torch

dataset = InRAMDataset(X, y, timestamps=timestamps, asset_names=asset_names)

X_sample, y_sample, timestamp_sample, asset_names = dataset[0]

assert isinstance(dataset, torch.utils.data.Dataset)
#assert len(dataset) == 10

assert torch.is_tensor(X_sample)
assert X_sample.shape ==  (n_channels, lookback, n_assets)

assert torch.is_tensor(y_sample)
assert y_sample.shape == (n_channels, horizon, n_assets)

assert timestamp_sample == timestamps[0]

In [40]:
from deepdow.data import RigidDataLoader

torch.manual_seed(1)
batch_size = 4

dataloader = RigidDataLoader(dataset, batch_size=batch_size)

for X_batch, y_batch, timestamps_batch, asset_names_batch in dataloader:
    print(X_batch.shape)
    print(y_batch.shape)
    print(asset_names_batch)
    print(list(map(str, timestamps_batch)))
    print()

torch.Size([4, 5, 90, 30])
torch.Size([4, 5, 30, 30])
['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']
['2014-03-19 00:00:00', '2021-06-04 00:00:00', '2018-03-13 00:00:00', '2006-12-21 00:00:00']

torch.Size([4, 5, 90, 30])
torch.Size([4, 5, 30, 30])
['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']
['2007-09-27 00:00:00', '2008-04-29 00:00:00', '2012-09-10 00:00:00', '2010-06-25 00:00:00']

torch.Size([4, 5, 90, 30])
torch.Size([4, 5, 30, 30])
['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT']
['2006-09-01 00:00:00', '2

In [1]:
import torch

# 创建一个 2x3 的张量
tensor = torch.arange(6)  # tensor([0, 1, 2, 3, 4, 5])
print("Original tensor:", tensor)
print("Original shape:", tensor.shape)  # torch.Size([6])

# 使用 view 转换为 2x3
tensor_2x3 = tensor.view(2, 3)
print("Reshaped to 2x3:\n", tensor_2x3)
print("New shape:", tensor_2x3.shape)  # torch.Size([2, 3])

Original tensor: tensor([0, 1, 2, 3, 4, 5])
Original shape: torch.Size([6])
Reshaped to 2x3:
 tensor([[0, 1, 2],
        [3, 4, 5]])
New shape: torch.Size([2, 3])


In [41]:
from deepdow.layers import CovarianceMatrix
import torch
from deepdow.layers import Conv
torch.manual_seed(3)

x = torch.rand(2, 7, 5, 3) * 100
layer = CovarianceMatrix(sqrt=False)

In [42]:
layer = Conv(n_input_channels=7,
             n_output_channels=1,
             kernel_size=3,
             method='2D')

In [43]:
layer(x)

tensor([[[[39.1130, 20.3913, 14.7150],
          [ 8.8522, 25.4509, 32.0081],
          [ 7.8578,  3.3538,  3.4022],
          [10.6471, 25.6159, 25.5635],
          [20.3321, 23.6899, 21.8828]]],


        [[[ 3.6374, 44.0102, 19.9943],
          [ 3.8684, 30.7076,  5.2100],
          [33.8895, 28.1290, 37.7850],
          [ 4.6121, 17.0600, 22.5956],
          [ 7.0132, 35.9687, -6.4219]]]], grad_fn=<ConvolutionBackward0>)

In [28]:
layer = Conv(n_input_channels=5,
             n_output_channels=1,
             kernel_size=3,
             method='1D')

In [30]:
y = layer(x)

In [6]:
import torch
import torch.nn as nn

class WeightedAverage(nn.Module):
    def __init__(self, num_tensors):
        super(WeightedAverage, self).__init__()
        # 定义可学习的权重参数，初始化为均匀分布
        self.weights = nn.Parameter(torch.ones(num_tensors) / num_tensors)
    
    def forward(self, tensor_list):
        # 确保权重归一化（和为1）
        normalized_weights = torch.softmax(self.weights, dim=0)
        # 对tensor_list进行加权平均
        weighted_sum = torch.stack(tensor_list).mul(normalized_weights.view(-1, 1, 1, 1, 1)).sum(dim=0)
        return weighted_sum

# 示例用法
# 假设有3个形状为 (batch_size, channels, height, width) 的张量
tensor_list = [
    torch.randn(4, 3, 32, 32),
    torch.randn(4, 3, 32, 32),
    torch.randn(4, 3, 32, 32)
]

# 初始化模型
model = WeightedAverage(num_tensors=len(tensor_list))

# 计算加权平均
result = model(tensor_list)

# 输出结果形状和学习到的权重
print("Output shape:", result.shape)
print("Learned weights:", torch.softmax(model.weights, dim=0))

Output shape: torch.Size([4, 3, 32, 32])
Learned weights: tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)


In [7]:
import torch

import torch.nn as nn

tensor = torch.arange(0, 30, dtype=torch.long)

dim = 10

emb = nn.Embedding(30, dim)

In [12]:
t = emb(tensor)

In [13]:
t.unsqueeze(0).unsqueeze(2)

tensor([[[[-9.2447e-01, -7.0041e-01, -4.9089e-01, -4.2083e-01, -2.0059e+00,
            2.1742e+00, -3.9666e-01,  2.9655e-01, -1.1133e+00,  8.5241e-01]],

         [[ 1.2099e+00, -2.5712e+00,  8.2204e-01, -5.8806e-01,  1.1271e+00,
           -2.1169e-01, -2.0741e-02, -7.7167e-02,  6.5165e-01,  4.1981e-01]],

         [[-4.5275e-02,  6.8487e-01, -1.4538e+00, -6.1737e-01, -1.0330e-01,
            1.9118e+00,  3.3566e-01,  1.9137e+00,  3.2555e-03, -5.3985e-01]],

         [[-1.8511e-02,  2.3246e-01, -2.1942e+00, -1.5171e+00, -1.6909e-01,
            3.6949e-01, -7.4641e-01,  9.4087e-02, -1.6966e-02,  1.7908e+00]],

         [[ 8.1399e-01,  1.3211e+00,  4.2165e-01,  8.5928e-01,  4.9151e-01,
            4.5821e-01, -1.5768e-01, -5.6570e-01, -1.2210e+00,  1.3644e+00]],

         [[ 1.1205e+00, -2.3709e+00, -4.9465e-02, -2.6575e-01,  3.2112e-01,
           -5.0124e-01,  7.2168e-01, -2.9508e-03,  1.1567e+00, -1.2307e+00]],

         [[ 3.4055e-01,  2.9073e-01, -2.2858e-01, -2.7354e-01, -4.8976

In [18]:
t.unsqueeze(0).unsqueeze(2).shape

torch.Size([1, 30, 1, 10])

In [19]:
emb(tensor).unsqueeze(0).unsqueeze(2).expand(32, dim, 10, 10)

RuntimeError: The expanded size of the tensor (10) must match the existing size (30) at non-singleton dimension 1.  Target sizes: [32, 10, 10, 10].  Tensor sizes: [1, 30, 1, 10]